In [19]:
%pip install opencv-python
%pip install mediapipe
%pip install pyautogui

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



In [20]:
import cv2
import mediapipe as mp
import pyautogui
import time

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
camera = cv2.VideoCapture(0)

def jari_terbuka(landmarks, hand_label):
    ibuJariTerbuka = False
    if hand_label == 'Right':
        ibuJariTerbuka = landmarks[4].x < landmarks[3].x
    elif hand_label == 'Left':
        ibuJariTerbuka = landmarks[4].x > landmarks[3].x

    telunjuk_terbuka = landmarks[8].y < landmarks[6].y
    tengah_terbuka = landmarks[12].y < landmarks[10].y
    manis_terbuka = landmarks[16].y < landmarks[14].y
    kelingking_terbuka = landmarks[20].y < landmarks[18].y

    count = sum([ibuJariTerbuka, telunjuk_terbuka, tengah_terbuka, manis_terbuka, kelingking_terbuka])
    return count, ibuJariTerbuka, telunjuk_terbuka, tengah_terbuka, manis_terbuka, kelingking_terbuka

# Cooldown timer
cooldown_time = 1.0  # seconds
last_action_time = 0
displayed_gesture = "WAITING..."  # Gesture to show on screen

with mp_hands.Hands(
    model_complexity=0,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as hands:
    while True:
        success, frame = camera.read()
        if not success:
            break

        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(frame_rgb)

        detected_gesture = "UNKNOWN"

        if result.multi_hand_landmarks:
            for idx, landmarks in enumerate(result.multi_hand_landmarks):
                hand_label = result.multi_handedness[idx].classification[0].label
                mp_drawing.draw_landmarks(frame, landmarks, mp_hands.HAND_CONNECTIONS)

                total, ibu, telunjuk, tengah, manis, kelingking = jari_terbuka(landmarks.landmark, hand_label)

                if total == 0 or (total == 1 and ibu):
                    detected_gesture = "DOWN"
                elif total == 1 and not ibu and not tengah:
                    detected_gesture = "LEFT"
                elif total == 2:
                    detected_gesture = "RIGHT"
                elif total in [4, 5]:
                    detected_gesture = "UP"
                elif total == 3:
                    detected_gesture = "SPACE"

        # Execute & display gesture only if cooldown is done
        now = time.time()
        if detected_gesture != "UNKNOWN" and now - last_action_time >= cooldown_time:
            if detected_gesture == "UP":
                pyautogui.press("up")
            elif detected_gesture == "DOWN":
                pyautogui.press("down")
            elif detected_gesture == "LEFT":
                pyautogui.press("left")
            elif detected_gesture == "RIGHT":
                pyautogui.press("right")
            elif detected_gesture == "SPACE":
                pyautogui.press("space")

            displayed_gesture = detected_gesture
            last_action_time = now

        # Show only executed gesture
        cv2.putText(frame, f"{displayed_gesture}", (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

        cv2.imshow('Gesture Control', frame)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

camera.release()
cv2.destroyAllWindows()
